# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Training Data & Feature views</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/electricity/3_feature_views_and_training_dataset.ipynb)



## 🗒️ This notebook is divided into 3 main sections:
1. **Feature Selection**,
2. **Feature preprocessing**,
3. **Training datasets creation**

![02_training-dataset](../../images/02_training-dataset.png)

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import pandas as pd

from functions import *

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

In [ ]:
citibike_usage_fg = fs.get_or_create_feature_group(
    name="citibike_usage",
    version=1
)

In [ ]:
citibike_stations_info_fg = fs.get_or_create_feature_group(
    name="citibike_stations_info",
    version=1
)

In [ ]:
us_holidays_fg = fs.get_or_create_feature_group(
    name="us_holidays",
    version=1
)

In [ ]:
meteorological_measurements_fg = fs.get_or_create_feature_group(
    name="meteorological_measurements",
    version=1
)

## <span style="color:#ff5f27;"> 🖍 Feature View Creation and Retrieving </span>

Let's start by selecting all the features you want to include for model training/inference.

In [ ]:
# Select features for training data.
fg_query = citibike_usage_fg.select_all()\
                                .join(
                                    us_holidays_fg.select_except(["timestamp"]), on="date"
                                )\
                                .join(
                                    meteorological_measurements_fg.select_except(["timestamp"]), on="date"
                                )

In [ ]:
# uncomment this if you would like to view query results
final_df = fg_query.read()

In [ ]:
final_df.head(5)

In [ ]:
final_df.shape

### <span style="color:#ff5f27;"> 🤖 Transformation Functions</span>

Hopsworks Feature Store provides functionality to attach transformation functions to feature views and comes with built-in transformation functions such as `min_max_scaler`, `standard_scaler`, `robust_scaler` and `label_encoder`.

You will preprocess our data using *min-max scaling* on numerical features and *label encoding* on categorical features. To do this you simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [ ]:
final_df.columns

In [ ]:
columns_to_transform = final_df.drop(columns=["date", "station_id"]).columns
columns_to_transform = columns_to_transform.tolist()

In [ ]:
# Map features to transformation functions.
transformation_functions = {col: fs.get_transformation_function(name="min_max_scaler") for col in columns_to_transform}

`Feature Views` stands between **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create **Feature Views** which store a metadata of our data. Having **Feature Views** we can create **Training Dataset**.

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.

In order to create Feature View we can use `FeatureStore.create_feature_view()` method.

We can specify next parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- our target variable.

- `transformation_functions` - functions to transform our features.

- `query` - query object with data.

In [ ]:
feature_view = fs.create_feature_view(
    name='citibike_feature_view',
    version=1,
    transformation_functions=transformation_functions,
    query=fg_query
)

For now `Feature View` is saved in Hopsworks and we can retrieve it using `FeatureStore.get_feature_view()`.

In [ ]:
feature_view = fs.get_feature_view(
    name = 'citibike_feature_view',
    version = 1
)

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

To create training dataset we use `FeatureView.create_training_data()` method.

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- We can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

- We can create **train, test** splits using `create_train_test_split()`. 

- We can create **train,validation, test** splits using `create_train_validation_test_splits()` methods.

- The only thing is that we should specify desired ratio of splits.

### <span style="color:#ff5f27;"> ⛳️ Train and test splits</span>

In [ ]:
# Create training dataset based event time filter
start_time = convert_date_to_unix("2022-01-01")
end_time = convert_date_to_unix("2022-01-20")


td_train_version, td_job = feature_view.create_training_data(
        start_time = start_time,
        end_time = end_time,    
        description = 'citibike training dataset 2022-01-01 -> 2022-01-20',
        data_format = "csv",
        coalesce = True,
        write_options = {'wait_for_job': True},
    )

In [ ]:
# Create test dataset based event time filter
start_time = convert_date_to_unix("2022-01-21")
end_time = convert_date_to_unix("2022-01-30")


td_train_version, td_job = feature_view.create_training_data(
        start_time = start_time,
        end_time = end_time,    
        description = 'citibike testing dataset 2022-01-21 -> 2022-01-30',
        data_format = "csv",
        coalesce = True,
        write_options = {'wait_for_job': True},
    )

---

## <span style="color:#ff5f27;">⏭️ **Next:** Part 04 </span>

In the next notebook you will train a model on the Training dataset, that was created in this notebook.